# ETL - Sucursales - **CHardy Tecno Store**
---
## 1. Extracción.
### 1.1. importamos el DataFrame Sucursaless.csv

In [ ]:
import pandas as pd
from pathlib import Path
import seaborn as sns

sns.set_theme(style="whitegrid")

# Ruta al archivo Parquet
ruta_archivo = Path("data/raw/sucursales.csv")

try:
    if not ruta_archivo.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {ruta_archivo.resolve()}")
    
    df = pd.read_csv(ruta_archivo)

    print(" Extracción correcta")
    print(f" Registros cargados: {len(df)}")
    print(f" Columnas: {len(df.columns)}")


except FileNotFoundError as e:
    print(f"❌ Error: {e}")
except Exception as e:
    print(f"⚠️ Error inesperado: {e}")

## 2. Transformación
### 2.1. Verificamos los Datos Extraidos

In [ ]:
df.info()

In [ ]:
df.head(30)

In [ ]:
# Filtrar duplicados (todas las columnas)
df_duplicados = df[df.duplicated()] # toda la fila debe estar duplicada
print(df_duplicados)

df_duplicados_id = df[df.duplicated(subset=["sucursal_id"], keep=False)]
print("sucursal_id:\n", df_duplicados_id)


In [ ]:
# Detectar duplicados por direccion + provincia
df_duplicados = df[df.duplicated(subset=["direccion_sucursal", "provincia_sucursal"], keep=False)]

print(">>> Duplicados detectados (direccion + provincia):")
print(df_duplicados)

# Control: mostrar agrupados por direccion/provincia con conteo
df_check = (
    df.groupby(["direccion_sucursal", "provincia_sucursal"])
      .size()
      .reset_index(name="conteo")
      .query("conteo > 1")
)

print("\n>>> Direcciones/provincias repetidas con conteo:")
print(df_check)


In [ ]:
# Marcar duplicados por direccion + provincia
df["es_duplicado_dirprov"] = df.duplicated(
    subset=["direccion_sucursal", "provincia_sucursal"], keep=False
)

# Control: mostrar las filas marcadas como duplicadas
print(">>> Sucursales marcadas como duplicadas (direccion + provincia):")
print(df[df["es_duplicado_dirprov"]])

# Control: resumen de direcciones/provincias repetidas
df_check = (
    df.groupby(["direccion_sucursal", "provincia_sucursal"])
      .size()
      .reset_index(name="conteo")
      .query("conteo > 1")
)

print("\n>>> Direcciones/provincias repetidas con conteo:")
print(df_check)


In [ ]:
print(df.isnull().sum()) # Muestra valores nulos (valor cero no es nulo)

In [ ]:
# Valores únicos de la columna 
valores_unicos = df["provincia_sucursal"].unique()
print(valores_unicos)

In [ ]:
# Elimina espacios al inicio y al final('Misiones ')
df["provincia_sucursal"] = df["provincia_sucursal"].str.strip()

# Reemplazar valores exactos en la columna 'provincia_sucursal'
df["provincia_sucursal"] = df["provincia_sucursal"].replace({
    "Bunoes Aires": "Buenos Aires",
    "Cordoba": "Córdoba"
})

# Eliminar filas donde 'provincia_sucursal' sea NaN
df = df.dropna(subset=["provincia_sucursal"])

In [ ]:
# Reviso los Cambios hechos
valores_unicos = df["provincia_sucursal"].unique()
print(valores_unicos)

print(df.isnull().sum()) # Muestra valores nulos, NaN (valor cero no es nulo)

## 3. Carga.
### 3.1. Guardamos la dimensión en formato parquet.

In [ ]:
import os

filename = "data/processed/dim_sucursales.parquet"

try:
    df.to_parquet(filename, index=False)
    if os.path.exists(filename):
        print(f"✔️ Dataset guardado con éxito en: {filename}")
    else:
        print("⚠️ No se encontró el archivo después de guardar.")
except Exception as e:
    print(f"❌ Error al guardar el dataset: {e}")